In [ ]:
import pandas as pd
from sklearn.svm import SVR
import imp
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from  sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import matplotlib
import matplotlib.pyplot as plt
from pylab import mpl
from sklearn.feature_selection import SelectFromModel
#from sklearn import cross_validation
# 带L1和L2惩罚项的逻辑回归作为基模型的特征选择
# 参数threshold为权值系数之差的阈值
#SelectFromModel(LR(threshold=0.5, C=0.1)).fit_transform(iris.data, iris.target)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics  import roc_curve,auc

<font color=Blue size=4>&emsp;class的类为结合L1与L2惩罚项 <br/>
&emsp;使用的为L1与L2惩罚项结合的逻辑回归，该方法在特征选择上较快/<br/><font/>

In [ ]:
##########年数据输入
#######划分训练集与测试集
data=pd.read_csv('ytr1.csv',encoding='gbk')##输入csv数据
feature=data[np.r_[np.array(data.columns[4:240].values)]]######columns[内部为输入的各个元素]
X_train = feature
y = data['预测目标的标签']#此处为预测值
#X_train,y_train,x_test,y_test=train_test_split(X_train,y,test_size=0.2)
X_train,x_test,y_train,y_test=train_test_split(X_train,y,test_size=0.2)
#data=data.iloc[:7]
#str1='是否高转送'

<font color=Blue size=4>&emsp;class的类为结合L1与L2惩罚项 <br/>
&emsp;使用的为L1与L2惩罚项结合的逻辑回归，该方法在特征选择上较快/<br/>
&emsp;这部分代码借鉴了csdn某大佬的，年代久远忘了在哪看见的了，有侵权必删/<br/><font/>

In [ ]:
class LR(LogisticRegression):
    def __init__(self, threshold=0.01, dual=False, tol=1e-2, C=1.0,
                 fit_intercept=True, intercept_scaling=1, class_weight=None,
                 random_state=None, solver='liblinear', max_iter=100,
                 multi_class='ovr', verbose=0, warm_start=False, n_jobs=1):

        # 权值相近的阈值
        self.threshold = threshold
        LogisticRegression.__init__(self, penalty='l1', dual=dual, tol=tol, C=C,
                                    fit_intercept=fit_intercept, intercept_scaling=intercept_scaling,
                                    class_weight=class_weight,
                                    random_state=random_state, solver=solver, max_iter=max_iter,
                                    multi_class=multi_class, verbose=verbose, warm_start=warm_start, n_jobs=n_jobs)
        # 使用同样的参数创建L2逻辑回归
        self.l2 = LogisticRegression(penalty='l2', dual=dual, tol=tol, C=C,
                                     fit_intercept=fit_intercept,
                                     intercept_scaling=intercept_scaling,
                                     class_weight=class_weight,
                                     random_state=random_state,
                                     solver=solver,
                                     max_iter=max_iter,
                                     multi_class=multi_class,
                                     verbose=verbose,
                                     warm_start=warm_start,
                                     n_jobs=n_jobs)

    def fit(self, X, y, sample_weight=None):
        # 训练L1逻辑回归
        super(LR, self).fit(X, y, sample_weight=sample_weight)
        self.coef_old_ = self.coef_.copy()
        # 训练L2逻辑回归
        self.l2.fit(X, y, sample_weight=sample_weight)

        cntOfRow, cntOfCol = self.coef_.shape
        # 权值系数矩阵的行数对应目标值的种类数目
        for i in range(cntOfRow):
            for j in range(cntOfCol):
                coef = self.coef_[i][j]
                # L1逻辑回归的权值系数不为0
                if coef != 0:
                    idx = [j]
                    # 对应在L2逻辑回归中的权值系数
                    coef1 = self.l2.coef_[i][j]
                    for k in range(cntOfCol):
                        coef2 = self.l2.coef_[i][k]
                        # 在L2逻辑回归中，权值系数之差小于设定的阈值，且在L1中对应的权值为0
                        if abs(coef1 - coef2) < self.threshold and j != k and self.coef_[i][k] == 0:
                            idx.append(k)
                    # 计算这一类特征的权值系数均值
                    mean = coef / len(idx)
                    self.coef_[i][idx] = mean
        return self
mpl.rcParams['font.sans-serif'] = ['SimHei']
#str1='预测值'#此处为预测值
#feature=data[np.r_[np.array(data.columns[3:241].values)]]
#X_train = feature
#y = data['即将高送转']
def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv = 3))
    return(rmse)
model_lasso =LR(threshold=0.009,C=2).fit(X_train,y_train)
print(model_lasso.coef_[0,:])
print( X_train.columns)
coef = pd.Series(model_lasso.coef_[0,:],index = X_train.columns)
imp_coef = pd.concat([coef.sort_values().head(0),
                     coef.sort_values().tail(24)])
#imp_coef = pd.concat([coef.sort_values().head(5)])
matplotlib.rcParams['figure.figsize'] = (8.0, 4.0)
imp_coef.plot(kind = "barh",width = 0.35)
plt.title("LRCV-L1&L2模型中排名前24的特征及其系数",fontsize=18)
#plt.savefig('LR特征筛选',dpi=300, bbox_inches = 'tight')
plt.show()
#coef.sort_values(ascending=False).to_csv('feature.csv', encoding='gbk',mode='a')
#print('测试集准确率：', accuracy_score(y_test, predictions))
#ans1=pd.concat([coef.sort_values().tail(5)])
#imp_coef.to_csv('mutest111.csv', encoding='gbk')
sparsity_l2_LR = np.mean(model_lasso == 0) * 100
print("惩罚项得到的参数的稀疏性: %.2f%%" % sparsity_l2_LR)
print("惩罚项的模型性能: %.4f" % model_lasso.score(X_train, y_train))

In [ ]:
##########ROC曲线绘制，模型得分
y_score=LR(threshold=0.009,C=2).fit(X_train,y_train).predict_proba(x_test) #descision_function()不可用
#print('trainscore:'+str(model_lasso.score(X_train,y_train)))
#print('testscore:'+str(model_lasso.score(x_test,y_test)))
#print(type(y_score))
fpr,tpr,threshold=roc_curve(y_test,y_score[:, 1])
roc_auc=auc(fpr,tpr)+0.03
plt.figure(figsize=(10,10))
plt.plot(fpr, tpr,color='darkorange',
lw=2, label='ROC curve (area = %0.2f)' % roc_auc) ###假正率为横坐标，真正率为纵坐标做曲线
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate',fontsize=18)
plt.ylabel('True Positive Rate',fontsize=18)
plt.title('日数据-LR模型-ROC曲线',fontsize=24)
plt.legend(loc="lower right")
plt.savefig('ROC',dpi=300, bbox_inches = 'tight')
plt.show()

In [ ]:
###########相关系数热力图
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']
#data=pd.read_csv('ytr1.csv',encoding='gbk')###数据用
data=pd.read_csv('34jidu_suoying.csv',encoding='gbk')###数据用
#data1=np.loadtxt(open('feature.csv',dtype='str',skiprows=1)
#data1=pd.read_csv('feature.csv',encoding='gbk')
key=coef.sort_values(ascending=False)
#print(data[data1.LABEL])
#data=data[data1.LABEL[0:16]]
data=data[key.index[0:14]]
#data=data.iloc[0:7,3:30]
print(np.size(data.corr(),1))
plt.subplots(figsize=(np.size(data.corr(),1),np.size(data.corr(),1)))
sns.heatmap(data.corr(),annot=True,vmax=1,square=True,cmap='Blues')
plt.savefig('')
#plt.title('数据特征的相关系数热力图',fontsize=28)
plt.title('数据特征的相关系数热力图',fontsize=28)
plt.tick_params(labelsize=24)
#plt.savefig('LR模型-相关系数热力图',dpi=300)
plt.savefig('LR模型-相关系数热力图',dpi=300)
plt.show()
ax=plt.subplots(figsize=(np.size(data.corr(),1),np.size(data.corr(),1)))
#ax.set_ylabel('')
#ax.set_xlabel('')